In [1]:
'''
用于对一次实验中的多路iFEMG信号进行特征提取

没有计算主动与拮抗肌肉之间的差值'''

import pandas as pd
import os

from gui_model_utils import form_feature_df
from iFEMG_feature import df_save_csv

In [33]:
'''
获取文件夹路径中的所有文件名
'''
folder_path = r"E:\Data\积水潭患者数据集\w2s2\healthy"
file_name_list = []
for file in os.listdir(folder_path):
    file_name_list.append(file.split('.')[0])
    pass
# print(file_name_list)
print(set(file_name_list))

{'bi-1', 'tri-1', 'w2s2_healthy_tricps_br', 'bi-2', 'tri-mvc'}


In [34]:
"""
init dataframe
REVISE BEFORE RUNNING:  
    1. ch_name_list
    2. ch_str
"""
feature_name_list = ['subject_name', 'height(cm)', 'weight(kg)', 'gender', 'age', 'sensor_channel', 'label(kg)', 
                    'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']

ch1_str = "agonist_ch1"
ch2_str = "antagonist_ch1"
ch3_str = "antagonist_ch2"

ch1_name_list = []
ch2_name_list = []
ch3_name_list = []
for i in feature_name_list:
    ch1_name_list.append(ch1_str)
    ch2_name_list.append(ch2_str)
    ch3_name_list.append(ch3_str)

all_feature_df = pd.DataFrame(columns=pd.MultiIndex.from_product([[ch1_str, ch2_str, ch3_str], feature_name_list]))
print(all_feature_df)
print("Init dataframe shape: ", all_feature_df.shape)

Empty DataFrame
Columns: [(agonist_ch1, subject_name), (agonist_ch1, height(cm)), (agonist_ch1, weight(kg)), (agonist_ch1, gender), (agonist_ch1, age), (agonist_ch1, sensor_channel), (agonist_ch1, label(kg)), (agonist_ch1, FMG_increase), (agonist_ch1, mav), (agonist_ch1, rms), (agonist_ch1, wave_length), (agonist_ch1, zero_crossing), (agonist_ch1, slope_sign_change), (agonist_ch1, mean_freq), (agonist_ch1, mean_power_freq), (antagonist_ch1, subject_name), (antagonist_ch1, height(cm)), (antagonist_ch1, weight(kg)), (antagonist_ch1, gender), (antagonist_ch1, age), (antagonist_ch1, sensor_channel), (antagonist_ch1, label(kg)), (antagonist_ch1, FMG_increase), (antagonist_ch1, mav), (antagonist_ch1, rms), (antagonist_ch1, wave_length), (antagonist_ch1, zero_crossing), (antagonist_ch1, slope_sign_change), (antagonist_ch1, mean_freq), (antagonist_ch1, mean_power_freq), (antagonist_ch2, subject_name), (antagonist_ch2, height(cm)), (antagonist_ch2, weight(kg)), (antagonist_ch2, gender), (antago

In [31]:
"""
read data
NEED TO CHANGE BEFORE RUNNING:
    1. file_name_list1 文件名
    2. label_list 数据的重量标签
    3. subject_arg_input 受试者的其他特征参数
    4. signal_channel_list 有用信号所在的通道号，ch_name_list 主动肌或拮抗肌，channel_name = "bicps_br"肌肉名称对应。三者需对应
    5. abandon_ms = 1000, 抛弃前一秒数据，尽量确保数据是等长收缩状态
"""
# file_name_list1 = ['bi-0', 'bi-05', 'bi-1', 'bi-2'] # 文件名
file_name_list1 = ['bi-1', 'bi-2'] # 文件名
label_list1 = [1, 2]   # 数据的重量标签
subject_name = "w2s1_healthy"

subject_arg_input = {"subject_height": 168,
                    "subject_weight": 75,
                    "subject_age": 38,
                    "subject_gender": 1,
                    "subject_name": subject_name}


# signal_channel, 信号对应的传感器通道
signal_channel_list = [1,3,4]

for i in range(len(file_name_list1)):
    #raw_data = pd.read_table(folder_path+'\\'+file_name_list1[i]+'.db', sep = ';', header = None)
    #label = read_label(folder_path+'\\'+file_name_list1[i]+'.txt')
    temp_df1 = form_feature_df(db_file_path=folder_path+'\\'+file_name_list1[i]+'.db',
                            time_file_path=folder_path+'\\'+file_name_list1[i]+'.txt',
                            signal_channel=signal_channel_list[0],
                            channel_name="bicps_br",
                            abandon_ms=1000,
                            strength_level=label_list1[i],
                            **subject_arg_input)
    temp_df1.columns=[ch1_name_list, feature_name_list]
    # medial长头
    temp_df2 = form_feature_df(db_file_path=folder_path+'\\'+file_name_list1[i]+'.db',
                            time_file_path=folder_path+'\\'+file_name_list1[i]+'.txt',
                            signal_channel=signal_channel_list[2],
                            channel_name="tricps_br_medial",
                            abandon_ms=1000,
                            strength_level=label_list1[i],
                            **subject_arg_input)
    temp_df2.columns=[ch3_name_list, feature_name_list]
    # lateral外侧头
    temp_df3 = form_feature_df(db_file_path=folder_path+'\\'+file_name_list1[i]+'.db',
                            time_file_path=folder_path+'\\'+file_name_list1[i]+'.txt',
                            signal_channel=signal_channel_list[1],
                            channel_name="tricps_br_lateral",
                            abandon_ms=1000,
                            strength_level=label_list1[i],
                            **subject_arg_input)
    temp_df3.columns=[ch2_name_list, feature_name_list]
    # 横向拼接三个通道特征dataframe
    temp_df = pd.concat([temp_df3, temp_df2, temp_df1], axis=1)
    print(temp_df.shape)
    # 数据加入all_feature_df
    all_feature_df = pd.concat([all_feature_df, temp_df], axis=0, ignore_index=True)
    print(all_feature_df.shape)
    pass

all_feature_df.head

(10, 45)
(10, 45)
(10, 45)
(20, 45)
(10, 45)
(30, 45)
(10, 45)
(40, 45)


<bound method NDFrame.head of       agonist_ch1                                                            \
     subject_name height(cm) weight(kg) gender age sensor_channel label(kg)   
0   w2s1_affected        168         75      1  38       bicps_br         0   
1   w2s1_affected        168         75      1  38       bicps_br         0   
2   w2s1_affected        168         75      1  38       bicps_br         0   
3   w2s1_affected        168         75      1  38       bicps_br         0   
4   w2s1_affected        168         75      1  38       bicps_br         0   
5   w2s1_affected        168         75      1  38       bicps_br         0   
6   w2s1_affected        168         75      1  38       bicps_br         0   
7   w2s1_affected        168         75      1  38       bicps_br         0   
8   w2s1_affected        168         75      1  38       bicps_br         0   
9   w2s1_affected        168         75      1  38       bicps_br         0   
10  w2s1_affected     

In [32]:
'''
写入文件，增加覆盖检查功能
运行前更改：
    1. 肌肉名称
'''
filename = folder_path+'\\'+subject_name+"_bicps_br.csv"

df_save_csv(all_feature_df, filename)

Done!


In [12]:
"""读取数据csv合并成一个"""
df_lpy1 = pd.read_csv(r"E:\Data\20230424-单人双次iFEMG肌力等级测试\lpy-1\lpy1_tricps_br.csv", index_col=0)
print(df_lpy1.shape)
df_lpy2 = pd.read_csv(r"E:\Data\20230424-单人双次iFEMG肌力等级测试\lpy-2\lpy2_tricps_br.csv", index_col=0)
print(df_lpy2.shape)

df_pym1 = pd.read_csv(r"E:\Data\20230424-单人双次iFEMG肌力等级测试\pym-1\pym1_tricps_br.csv", index_col=0)
print(df_pym1.shape)
df_pym2 = pd.read_csv(r"E:\Data\20230424-单人双次iFEMG肌力等级测试\pym-2\pym2_tricps_br.csv", index_col=0)
print(df_pym2.shape)

df_zk2 = pd.read_csv(r"E:\Data\20230424-单人双次iFEMG肌力等级测试\zk-2\zk2_tricps_br.csv", index_col=0)
print(df_zk2.shape)


(32, 45)
(31, 45)
(31, 45)
(32, 45)
(31, 45)


In [16]:
dd = pd.concat([df_lpy1, df_lpy2, df_pym1, df_pym2, df_zk2], ignore_index=True)
dd.shape

(157, 45)

In [17]:
df_save_csv(dd, r"E:\Data\20230424-单人双次iFEMG肌力等级测试\tricps_br(pure relaxed).csv")

Done!


In [18]:
# 定义df1
df1 = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': [7, 8, 9]})
df1.columns = pd.MultiIndex.from_tuples([('A', 'a'), ('A', 'b'), ('B', 'c')])
df1

A     B
   a  b  c
0  1  4  7
1  2  5  8
2  3  6  9

In [ ]:
# 定义df2
df2 = pd.DataFrame({'d': [10, 11, 12], 'e': [13, 14, 15], 'f': [16, 17, 18]})
df2.columns = pd.MultiIndex.from_tuples([('C', 'd'), ('C', 'e'), ('D', 'f')])

In [6]:
"""
3 channel model training
"""
import numpy as np
import time
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [8]:
data_df = pd.read_csv(r"E:\Data\20230424-单人双次iFEMG肌力等级测试\tricps_br(pure relaxed).csv", header=[0, 1], index_col=0)
print(data_df.shape)
data_df.head()

(156, 45)


,agonist_ch1,agonist_ch1.1,agonist_ch1.2,agonist_ch1.3,agonist_ch1.4,agonist_ch1.5,agonist_ch1.6,agonist_ch1.7,agonist_ch1.8,agonist_ch1.9,agonist_ch1.10,agonist_ch1.11,agonist_ch1.12,agonist_ch1.13,agonist_ch1.14,agonist_ch2,agonist_ch2.1,agonist_ch2.2,agonist_ch2.3,agonist_ch2.4,agonist_ch2.5,agonist_ch2.6,agonist_ch2.7,agonist_ch2.8,agonist_ch2.9,agonist_ch2.10,agonist_ch2.11,agonist_ch2.12,agonist_ch2.13,agonist_ch2.14,antagonist_ch1,antagonist_ch1.1,antagonist_ch1.2,antagonist_ch1.3,antagonist_ch1.4,antagonist_ch1.5,antagonist_ch1.6,antagonist_ch1.7,antagonist_ch1.8,antagonist_ch1.9,antagonist_ch1.10,antagonist_ch1.11,antagonist_ch1.12,antagonist_ch1.13,antagonist_ch1.14
,subject_name,height(cm),weight(kg),gender,age,sensor_channel,label(kg),FMG_increase,mav,rms,wave_length,zero_crossing,slope_sign_change,mean_freq,mean_power_freq,subject_name,height(cm),weight(kg),gender,age,sensor_channel,label(kg),FMG_increase,mav,rms,wave_length,zero_crossing,slope_sign_change,mean_freq,mean_power_freq,subject_name,height(cm),weight(kg),gender,age,sensor_channel,label(kg),FMG_increase,mav,rms,wave_length,zero_crossing,slope_sign_change,mean_freq,mean_power_freq
0.0,Li Peiyang,182,82,1,21,tricps_br_medial,0,-0.1591058255666322,-0.4189192336573084,-0.49395191996018273,-0.23903168535637156,0.24037658069188333,0.12169291968033438,24.60403577351601,47.82830333699141,Li Peiyang,182,82,1,21,tricps_br_lateral,0,0.020323402885805773,-0.020176660133639286,-0.01880180716640243,-0.010300544711808317,0.011105890786909317,-0.03595522036677728,220.15094172923506,228.97415824401037,Li Peiyang,182,82,1,21,bicps_br,0,-0.0061580781957695915,-0.41048329032373704,-0.48521495155729044,-0.21343477457061122,0.054837334292989856,0.13465125260812846,24.293232518332196,52.21386157784752
1.0,Li Peiyang,182,82,1,21,tricps_br_medial,0,-0.20504313384304704,1.5391137562878285,3.3223070323802615,0.4350703627283898,-0.24689518310458614,0.24001858195876738,19.981356308421997,29.222246354143522,Li Peiyang,182,82,1,21,tricps_br_lateral,0,0.04349068884860785,-0.02742116669600586,-0.027441425767020498,-0.020142703897837365,-0.022234876424075452,0.023088758204399696,233.1403757232035,231.0916177319488,Li Peiyang,182,82,1,21,bicps_br,0,-0.03354450075864014,6.506753937183124,9.050086457349764,2.963948779279438,-0.3553287715947758,0.09665832036855884,21.23411418684809,31.257801449863848
2.0,Li Peiyang,182,82,1,21,tricps_br_medial,0,-0.2761631201275767,-0.2269370566206912,-0.1920596865593652,-0.16467271222891028,0.0800507228494709,0.34669106566643126,50.55863790337148,53.37638062770314,Li Peiyang,182,82,1,21,tricps_br_lateral,0,0.010830403305834633,-0.018578121184808824,-0.018375027165499193,-0.033560973365053516,-0.01689341662198059,-0.008098246537679715,201.65585967201758,222.84021149869352,Li Peiyang,182,82,1,21,bicps_br,0,-0.017856586481450266,0.023037332265149,0.025014160959113643,0.007926303716033185,0.009602969939330613,-0.13542847162927893,52.22380980497939,55.863455090104274
3.0,Li Peiyang,182,82,1,21,tricps_br_medial,0,-0.16088538446599104,-0.14571575569869197,-0.1418560372569347,-0.08766262780195937,0.10378939971893193,0.19696316313016807,50.042295602992425,51.391275904653405,Li Peiyang,182,82,1,21,tricps_br_lateral,0,0.031135210060950092,0.0050430212643442374,0.01004052412915269,0.007743840981870412,-0.021996385858306346,0.01731154221517349,227.42428697681112,232.89792420017338,Li Peiyang,182,82,1,21,bicps_br,0,0.0009813845681335637,0.06527413840350052,0.08497442106220665,0.028375040175432964,-0.007829753061634196,-0.11063293527452515,52.151105137501126,55.24757121155364
4.0,Li Peiyang,182,82,1,21,tricps_br_medial,0,-0.15678558651494467,-0.18624132591716006,-0.1995958912117606,-0.09921872956948478,0.046024860331325794,0.15275610752951208,51.17919046635435,53.24067701378354,Li Peiyang,182,82,1,21,tricps_br_lateral,0,0.036353679340150914,-0.030066173897841494,-0.041254339123725045,-0.03024938380844307,0.023277299770008773,-0.005821342840752875,206.59848999129588,223.0

In [9]:
x1_data = data_df["agonist_ch1"][['height(cm)', 'weight(kg)', 'gender', 'age', 'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values
x2_data = data_df["agonist_ch2"][['height(cm)', 'weight(kg)', 'gender', 'age', 'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values
x3_data = data_df["antagonist_ch2"][['height(cm)', 'weight(kg)', 'gender', 'age', 'FMG_increase', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']].values

y_data = data_df["agonist_ch1"]['label(kg)'].values
x_data = np.concatenate((x1_data, x2_data, x3_data), axis=1)
print("data shape: ", x_data.shape)

train_data_r, test_data_r, train_label, test_label = train_test_split(x_data, y_data, test_size = 0.2, random_state = 0)
# train_data = preprocessing.normalize(train_data_r, norm = 'max')
# test_data = preprocessing.normalize(test_data_r, norm = 'max')
scaler = preprocessing.StandardScaler().fit(train_data_r)
train_data = scaler.transform(train_data_r)
test_data = scaler.transform(test_data_r)

print("train data shape: ", train_data.shape)
print("train label length: ", train_label.shape)
print("test data shape: ", test_data.shape)
print("test label length: ", test_label.shape)

KeyError: "None of [Index(['height(cm)', 'weight(kg)', 'gender', 'age', 'FMG_increase', 'mav',\n       'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq',\n       'mean_power_freq'],\n      dtype='object')] are in the [columns]"

In [ ]:
"""
Grid searching best parameters of SVR model
"""
# 记录开始训练时间
start_time = time.perf_counter()

# 自动选择合适的参数
svr = GridSearchCV(SVR(), param_grid={"kernel": ("rbf", "linear"), "C": np.logspace(-3, 3, 7), "gamma": np.logspace(-3, 3, 7)}, n_jobs=-1)
svr.fit(train_data, train_label)

print(svr.best_params_)

end_time = time.perf_counter()
print("training time(min): ", (end_time - start_time)/60)

start training, time:  2022--11--21 21:22:39
{'C': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}
finish, time:  2022--11--21 21:22:40


In [ ]:
score_test = svr.score(test_data, test_label)
score_train = svr.score(train_data, train_label)
print(str(svr) + "train score： " + str(score_train))
print(str(svr) + "test score： " + str(score_test))
test_pre = svr.predict(test_data)
train_pre = svr.predict(train_data)
print("train mean squared error: ", mean_squared_error(train_label, train_pre))
print("test mean squared error: ", mean_squared_error(test_label, test_pre))


GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'kernel': ('rbf', 'linear')})train score： 0.9492612112237954
GridSearchCV(estimator=SVR(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'kernel': ('rbf', 'linear')})test score： 0.9475451019066321
train mean squared error:  0.008495207450243339
test mean squared error:  0.006362872253929249
